In [1]:
# MLflow Training
def train_elasticnet(N_alpha, N_rho):
    import os
    import warnings
    import sys

    import pandas as pd
    import numpy as np
    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
    from sklearn.model_selection import train_test_split
    from sklearn.linear_model import ElasticNet

    import mlflow
    import mlflow.sklearn
    
    import logging
    logging.basicConfig(level=logging.WARN)
    logger = logging.getLogger(__name__)
    
    from data_processing import data_process

    def eval_metrics(actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        r2_test = r2_score(actual, pred)
        return rmse, r2_test


    warnings.filterwarnings("ignore")
    np.random.seed(40)

#     # Read the file
#     try:
#         df_raw = pd.read_csv('train.csv',index_col=0)
#     except Exception as e:
#         logger.exception(
#             "Unable to download training & test CSV, check your internet connection. Error: %s", e)
        
    # Data processing.
    df_processed = data_process()
    
    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(df_processed)

    # The predicted column is "SalePrice" .
    train_x = train.drop(["SalePrice"], axis=1)
    test_x = test.drop(["SalePrice"], axis=1)
    train_y = train[["SalePrice"]]
    test_y = test[["SalePrice"]]

    # Set default values if no N_alpha is provided
    if int(N_alpha) is None:
        N_alpha = 50
    else:
        N_alpha = int(N_alpha)

    # Set default values if no N_rho is provided
    if int(N_rho) is None:
        N_rho = 11
    else:
        N_rho = int(N_rho)
    
    alphaRange = np.logspace(-3, -1, N_alpha)
    rhoRange   = np.linspace(0,0.4, N_rho) # we avoid very small rho by starting at 0.1
    scores     = np.zeros((N_rho, N_alpha))
      
    
    # Execute ElasticNet
    for alphaIdx, alpha in enumerate(alphaRange):
        for rhoIdx, rho in enumerate(rhoRange):
            with mlflow.start_run():
                lr = ElasticNet(alpha=alpha, l1_ratio=rho, normalize=False)
                lr.fit(train_x, train_y)
                r2_train = lr.score(train_x, train_y)
        
        # Training Model Performances Evaluate Metrics
#         predicted_qualities = lr.predict(test_x)
#         (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)        

        # Evaluate Metrics
                predicted_qualities = lr.predict(test_x)
                (rmse, r2_test) = eval_metrics(test_y, predicted_qualities)

        # Print out metrics
                print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, rho))
                print("  RMSE: %s" % rmse)
                print("  Train R2: %s" % r2_train)
                print("  Test R2: %s" % r2_test)

        # Log parameter, metrics, and model to MLflow
                mlflow.log_param("model", 'elasticnet')
                mlflow.log_param("alpha", alpha)
                mlflow.log_param("l1_ratio", rho)
                mlflow.log_metric("rmse", rmse)
                mlflow.log_metric("r2_test", r2_test)
                mlflow.log_metric("r2_train", r2_train)
                #mlflow.log_metric("mae", mae)

                mlflow.sklearn.log_model(lr, "model")

In [2]:
train_elasticnet(30,11)

Elasticnet model (alpha=0.001000, l1_ratio=0.000000):
  RMSE: 0.10603442082756895
  Train R2: 0.948723686380187
  Test R2: 0.9340260316005999
Elasticnet model (alpha=0.001000, l1_ratio=0.040000):
  RMSE: 0.10468283092919625
  Train R2: 0.9476117001751545
  Test R2: 0.9356972141870353
Elasticnet model (alpha=0.001000, l1_ratio=0.080000):
  RMSE: 0.1039805609206908
  Train R2: 0.9463462791923181
  Test R2: 0.9365570771743933
Elasticnet model (alpha=0.001000, l1_ratio=0.120000):
  RMSE: 0.10378596015196268
  Train R2: 0.9452315187718907
  Test R2: 0.9367943232243552
Elasticnet model (alpha=0.001000, l1_ratio=0.160000):
  RMSE: 0.10380739312853685
  Train R2: 0.9443239070517755
  Test R2: 0.9367682151521904
Elasticnet model (alpha=0.001000, l1_ratio=0.200000):
  RMSE: 0.10405682513230399
  Train R2: 0.9435189441849268
  Test R2: 0.9364639790258086
Elasticnet model (alpha=0.001000, l1_ratio=0.240000):
  RMSE: 0.10444840708655084
  Train R2: 0.9427317979065419
  Test R2: 0.9359848874901124
E

Elasticnet model (alpha=0.002212, l1_ratio=0.120000):
  RMSE: 0.10457013184968839
  Train R2: 0.9418287656354748
  Test R2: 0.9358355933998315
Elasticnet model (alpha=0.002212, l1_ratio=0.160000):
  RMSE: 0.10484548437733472
  Train R2: 0.9404843049322209
  Test R2: 0.9354972349686816
Elasticnet model (alpha=0.002212, l1_ratio=0.200000):
  RMSE: 0.10512211741972562
  Train R2: 0.9393071080122892
  Test R2: 0.9351564070111765
Elasticnet model (alpha=0.002212, l1_ratio=0.240000):
  RMSE: 0.10560787665595509
  Train R2: 0.9381793925736668
  Test R2: 0.9345557503565214
Elasticnet model (alpha=0.002212, l1_ratio=0.280000):
  RMSE: 0.10597801510575561
  Train R2: 0.937264789996518
  Test R2: 0.9340962035219759
Elasticnet model (alpha=0.002212, l1_ratio=0.320000):
  RMSE: 0.10640843021659824
  Train R2: 0.9365000646467404
  Test R2: 0.9335597980775444
Elasticnet model (alpha=0.002212, l1_ratio=0.360000):
  RMSE: 0.10686638891369439
  Train R2: 0.9357761864500039
  Test R2: 0.9329866791329136


KeyboardInterrupt: 